# ChessVision Model Deployment

First follow the instructions in the [Data Processing Wiki](https://github.com/gudbrandtandberg/ChessVision/wiki/Data-Processing)

Then use this notebook to:

- Upload new model weights to S3

- (Optional) If `container_endpoint.py` changes, rebuild and push the container image to ECR.

- When new weights are in, redeploy endpoint:
  + Create new model
  + Create new endpoint config
  + Update endpoint

## Upload models archive to S3

In [17]:
%%sh 

s3_archive=s3://chessvision-bucket/model_archives/models.tar.gz
weights_dir=$CVROOT/weights/models
zip_file=$CVROOT/weights/models.tar.gz

# Zip the local files
tar -zcvf $zip_file $weights_dir

# And send them to s3
aws s3 cp $zip_file $s3_archive

upload: ../../weights/models.tar.gz to s3://chessvision-bucket/model_archives/models.tar.gz


tar: Removing leading '/' from member names
a Users/gudbrand/Programming/Chess/ChessVision/weights/best_models
a Users/gudbrand/Programming/Chess/ChessVision/weights/best_models/best_extractor.hdf5
a Users/gudbrand/Programming/Chess/ChessVision/weights/best_models/best_classifier.hdf5


### Build and push the container to ECR (can skip)

In [15]:
# Only necessary container_endpoint changes
#./build_and_push.sh

## Update sagemaker model

This operation also pulls new model artifacts from S3

In [40]:
import boto3
client = boto3.client('sagemaker')

In [45]:

#client.delete_model(
#    ModelName='chessvision-model'
#)

client.create_model(
    ModelName='chessvision-model',
    PrimaryContainer={
        'ContainerHostname': 'chessvision-algo',
        'Image': '580857158266.dkr.ecr.eu-central-1.amazonaws.com/chessvision-algo:latest',
        'ModelDataUrl': "s3://chessvision-bucket/model_archives/models.tar.gz",
    },
    ExecutionRoleArn='arn:aws:iam::580857158266:role/service-role/AmazonSageMaker-ExecutionRole-20190106T201024',
    EnableNetworkIsolation=False
)

{'ModelArn': 'arn:aws:sagemaker:eu-central-1:580857158266:model/chessvision-model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '82',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 03 Mar 2019 00:14:16 GMT',
   'x-amzn-requestid': '06681670-1130-457e-a116-53ac5b750be5'},
  'HTTPStatusCode': 200,
  'RequestId': '06681670-1130-457e-a116-53ac5b750be5',
  'RetryAttempts': 0}}

## Update endpoint to use the new model

In [46]:
client.delete_endpoint_config(
    EndpointConfigName='chessvision-endpoint-config'
)

config_response = client.create_endpoint_config(
    EndpointConfigName='chessvision-endpoint-config',
    ProductionVariants=[
        {
            'VariantName': 'chessvision-variant',
            'ModelName': 'chessvision-model',
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium',
            'InitialVariantWeight': 1
        },
    ])

In [ ]:
client.update_endpoint(
    EndpointName='chessvision-endpoint',
    EndpointConfigName='chessvision-endpoint-config'
)

## Test the endpoint

In [48]:
import boto3
import json
import requests
from cv_globals import CVROOT
local = False

with open(CVROOT + "/local_test/test.json", "r") as f:
    payload = f.read()

payload
    
client = boto3.client('sagemaker-runtime')
endpoint_name = "chessvision-endpoint"                      
content_type = "application/json"                           
accept = "text/plain"                                   

response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

print(response["Body"].read())

b'{"FEN": "q1r1r1k1/pb1nbppp/1p1ppn2/8/2PP4/1P1B1NN1/PB2QPPP/2R1R1K1"}'
